In [20]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from tqdm.notebook import tqdm
import random
import json
from time import sleep
import pygame

# Жанры

In [21]:

# catalog= requests.get('https://www.biblio-globus.ru/catalog/categories')
# soup=BeautifulSoup(catalog.text,'html.parser')
# cats=soup.find_all('li',class_="list-group-item")

# #
# #cats.find_all('a')['href']

In [22]:
# mega_genres_url=[]
# for cat in cats:
#     mega_genres_url.append(cat.find('a',attrs={'href':True})['href'])
# mega_genres_url

In [23]:
# mega_genres_url = mega_genres_url[:len(mega_genres_url)//2]

In [24]:
# for m in mega_genres_url:
#     print(m[m.rfind('/')+1:])

In [25]:
# genres=[]
# save=10
# for i in tqdm(range(len(mega_genres_url))):
#     mega_genre='https://www.biblio-globus.ru'+mega_genres_url[i]
#     sleep(random.random()/2)
#     mega= requests.get(mega_genre)
#     soup=BeautifulSoup(mega.text,'html.parser')
#     cats=soup.find_all('a',attrs={'class':'nav-link','style':"padding-left:14px;"})
#     if len(cats)==0:
#         if save<=0:
#             save=10
#             pd.Series(genres).to_csv('genres_url.csv')
#         genres.append(mega_genre)
#         save-=1
#     else:
#         for cat in tqdm(cats):
#             if save<=0:
#                 save=10
#                 pd.Series(genres).to_csv('genres_url.csv')
#             genres.append(cat['href'])
#             save-=1

# pygame.init()
# pygame.mixer.music.load('hypa_bd.mp3')
# pygame.mixer.music.play()
# pd.Series(genres).to_csv('genres_url.csv')
# sleep(10)
# pygame.mixer.music.stop()


In [26]:
genres=pd.read_csv('genres_url.csv',index_col=0)
genres=genres['0'].to_list()
genres=genres[:480]

In [27]:
# errs=[]
# for i,genre in enumerate(genres):
#     if genre.find('www')+1:
#         errs.append(i)

In [28]:
# # for err in errs:
#     #print(genres[err])
#     genres[err]=genres[err][genres[err].find('.ru')+3:]

In [29]:
# mega= requests.get('https://www.biblio-globus.ru/catalog/index/6182')
# # soup=BeautifulSoup(mega.text,'html.parser')
# cats=soup.find_all('a',attrs={'class':'nav-link','style':"padding-left:14px;"})

In [30]:
genres_idx=[]
for genre in genres:
    genres_idx.append(genre[genre.rfind('/')+1:])

In [ ]:
genres_idx

In [32]:
# pd.Series(mega_genres_url).to_csv('genres_url.csv')

In [33]:
biblio_df = pd.DataFrame(columns=['page_url','image_url','author','title','annotation','genre'])

# Функции

In [34]:
def get_all_page_books(page_url):
    books=requests.get(page_url)
    #print(books)
    soup = BeautifulSoup(books.text, 'html.parser')
    all_page_books=soup.find_all('a',class_='img_link')
    book_links=[]
    for book in all_page_books:
        book_links.append('https://www.biblio-globus.ru/'+book.get_attribute_list('href')[0])
    return book_links

In [103]:
def get_pages(page_url):
    page=requests.get(page_url)
    
    soup = BeautifulSoup(page.text, 'html.parser')
    # print(soup)
    nextest=soup.find_all('a',attrs={'aria-label':'Next'})[-1]['href']

    return int(nextest[nextest.find('e=')+2:nextest.find('&s')])

In [132]:
def get_dic_from_page(book_link):
    book_page=requests.get(book_link)#Обращаемся к странице
    book_html= BeautifulSoup(book_page.text, 'html.parser') #Получаем суп
    try:
        dic_json=json.loads(book_html.find('script').text)
        annotation=book_html.find(attrs={'property': 'og:description'})
        biblio_dic = {}

        biblio_dic['title'] =dic_json['name']
        biblio_dic['page_url'] =dic_json['offers']['url']
        biblio_dic['image_url'] =dic_json['image']
        biblio_dic['author'] =dic_json['author']['name']
        biblio_dic['annotation'] =annotation.get('content')
        biblio_dic['genre'] =dic_json['genre']
    except:
        print(book_link)
        return {}
    return biblio_dic

In [36]:
class Page:
    def __init__(self,url) -> None:
        self.url = url
    def __str__(self) -> str:
        return self.url
    def __add__(self,num):
        if isinstance(num, (int, float)):
            page_num=int(self.url[self.url.find('page=')+5:self.url.rfind('&sort')])+num
            return Page(self.url[:self.url.find('page=')+5]+str(page_num)+self.url[self.url.rfind('&sort'):])
           
        else:
            return NotImplemented  
        

In [130]:
def cat_parser(startpage,num_pages=100,saves=100,num_books=1000000):
    true_num=get_pages(str(startpage))
    biblio_df=pd.DataFrame(columns=['page_url','image_url','author','title','annotation','genre'])
    save_mark=saves
    for i in tqdm(range(min(num_pages,true_num)),'Pages'):
        # if i == true_num:
        #     break
        book_links=get_all_page_books(str(startpage+i))#Получаем массив с ссылками на все книги страницы
        # if len(book_links)==0:
        #     break
        for book_link in tqdm(book_links,'Books',leave=False):
            sleep(random.random()/2)#Чутка спим
            book=[get_dic_from_page(book_link)]#
           
            biblio_df=pd.concat([biblio_df,pd.DataFrame(book)],ignore_index=True)#Превращаем книгу в словарь
            if save_mark<=0:
                biblio_df.to_csv('books_data.csv',index=False,sep='\t',mode='a')
                display(biblio_df)
                biblio_df=pd.DataFrame(columns=['page_url','image_url','author','title','annotation'])
                save_mark=saves
            save_mark-=1
            num_books-=len(book)
            if num_books<=0:
                return biblio_df
    biblio_df.to_csv('books_data.csv',index=False,sep='\t',mode='a')            
    return biblio_df

    


In [135]:
def parser(num_pages=100):

    for genre_id in tqdm(genres_idx[3:],'Categories'):
        firstpage=f'https://www.biblio-globus.ru/catalog/category?id={genre_id}&page=1&sort=1&instock=&isdiscount='
        print(firstpage)
        cat_parser(Page(firstpage))


In [ ]:
cat_parser(Page(f'https://www.biblio-globus.ru/catalog/category?id={genres_idx[2]}&page=53&sort=1&instock=&isdiscount='))


# Парсинг

In [ ]:
#get_pages('https://www.biblio-globus.ru/catalog/category?id=241&page=1&sort=1&instock=&isdiscount=')

In [136]:
parser()
pygame.init()
pygame.mixer.music.load('hypa_bd.mp3')
pygame.mixer.music.play()
#pd.Series(genres).to_csv('genres_url.csv')
sleep(10)
pygame.mixer.music.stop()

Categories:   0%|          | 0/477 [00:00<?, ?it/s]

https://www.biblio-globus.ru/catalog/category?id=242&page=1&sort=1&instock=&isdiscount=


Pages:   0%|          | 0/100 [00:00<?, ?it/s]

Books:   0%|          | 0/12 [00:00<?, ?it/s]

Books:   0%|          | 0/12 [00:00<?, ?it/s]

...

Books:   0%|          | 0/12 [00:00<?, ?it/s]

Books:   0%|          | 0/12 [00:00<?, ?it/s]

Books:   0%|          | 0/12 [00:00<?, ?it/s]

Books:   0%|          | 0/12 [00:00<?, ?it/s]

,page_url,image_url,author,title,annotation,genre
0,https://www.biblio-globus.ru/product/10545564,https://static1.bgshop.ru/imagehandler.ashx?wi...,Верн Ж.,"Матиас Шандор, или Новый граф Монте-Кристо",Историко-приключенческий роман «Матиас Шандор»...,Исторические романы
1,https://www.biblio-globus.ru/product/10545558,https://static1.bgshop.ru/imagehandler.ashx?wi...,Ле Фаню Дж. Ш.,"В тусклом стекле: повести, рассказы",Джозеф Шеридан Ле Фаню – выдающийся писатель В...,Исторические романы
2,https://www.biblio-globus.ru/product/10545214,https://static1.bgshop.ru/imagehandler.ashx?wi...,Апухтин А. Н.,Сочинения. Проза,"Апухтин Алексей Николаевич - классик, которого...",Исторические романы
3,https://www.biblio-globus.ru/product/10542299,https://static1.bgshop.ru/imagehandler.ashx?wi...,Маккалоу К.,Битва за Рим,«Битва за Рим» – второй из цикла романов Колин...,Исторические романы
4,https://www.biblio-globus.ru/product/10534049,https://static1.bgshop.ru/imagehandler.ashx?wi...,Чуковский Н.,Балтийское небо,К 75-ЛЕТИЮ ПОЛНОГО СНЯТИЯ БЛОКАДЫ ЛЕНИНГРАДА. ...,Исторические романы
...,...,...,...,...,...,...
95,https://www.biblio-globus.ru/product/10462005,https://static1.bgshop.ru/imagehandler.ashx?wi...,Гончаров И.,Полное собрание романов в одном томе,В одном томе собраны все романы классика русск...,Исторические романы
96,https://www.biblio-globus.ru/product/10456680,https://static1.bgshop.ru/imagehandler.ashx?wi...,Бунин И.,Полное собрание рассказов в одном томе,В одном томе собраны все рассказы знаменитого ...,Исторические романы
97,https://www.biblio-globus.ru/product/10455625,https://static1.bgshop.ru/imagehandler.ashx?wi...,Бронте Э.,Грозовой перевал,Перед нами единственный роман английской писат...,Исторические романы
98,https://www.biblio-globus.ru/product/10455621,https://static1.bgshop.ru/imagehandler.ashx?wi...,Твен М.,Приключения Гекльберри Финна,Приключения Гекльберри Финна - одна из самых и...,Исторические романы


In [95]:
genres_idx

['226',
 '227',
 '241',
 '242',
 '248',
 '250',
 '251',
 '6168',
 '6169',
 '6170',
 '6171',
 '262',
 '263',
 '6173',
 '6174',
 '6176',
 '6177',
 '6178',
 '6179',
 '6180',
 '6181',
 '6182',
 '6183',
 '6184',
 '6186',
 '6187',
 '6188',
 '6189',
 '6112',
 '6113',
 '6114',
 '6115',
 '6116',
 '6117',
 '6118',
 '6119',
 '6120',
 '6121',
 '6122',
 '6129',
 '6130',
 '6131',
 '6132',
 '6133',
 '6134',
 '6135',
 '6136',
 '6137',
 '6138',
 '6140',
 '6141',
 '6142',
 '6143',
 '6144',
 '6146',
 '6147',
 '6148',
 '6149',
 '6150',
 '6151',
 '6153',
 '6154',
 '6155',
 '6157',
 '6158',
 '6159',
 '6160',
 '6161',
 '6162',
 '6164',
 '6165',
 '6166',
 '6167',
 '2422',
 '2464',
 '2465',
 '2466',
 '2467',
 '2468',
 '2470',
 '2471',
 '2473',
 '2474',
 '2475',
 '2476',
 '2477',
 '2479',
 '2486',
 '2488',
 '2489',
 '2491',
 '2492',
 '2493',
 '2495',
 '2497',
 '2498',
 '2500',
 '2502',
 '5887',
 '2508',
 '2509',
 '2510',
 '5889',
 '5890',
 '6091',
 '6092',
 '6093',
 '6094',
 '6095',
 '6096',
 '6097',
 '6098',
 

In [75]:
pg=get_pages(f'https://www.biblio-globus.ru/catalog/category?id={genres_idx[0]}&page=245&sort=0&instock=&isdiscount=')
pg

250

In [69]:
next=pg.find_all('a',attrs={'aria-label':'Next'})[-1]['href']

'250'

In [59]:
#cat_parser(Page(f'https://www.biblio-globus.ru/catalog/category?id={genres_idx[0]}&page=245&sort=0&instock=&isdiscount='))

Pages:   0%|          | 0/250 [00:00<?, ?it/s]

Books:   0%|          | 0/12 [00:00<?, ?it/s]

Books:   0%|          | 0/12 [00:00<?, ?it/s]

Books:   0%|          | 0/12 [00:00<?, ?it/s]

Books:   0%|          | 0/12 [00:00<?, ?it/s]

Books:   0%|          | 0/12 [00:00<?, ?it/s]

Books:   0%|          | 0/2 [00:00<?, ?it/s]

,page_url,image_url,author,title,annotation,genre
0,https://www.biblio-globus.ru/product/10934269,https://static1.bgshop.ru/imagehandler.ashx?wi...,Толстой А.Н.,"Петр I. В 3 т., Т.1.",Смерть молодого царя Федора Алексеевича могла ...,Классическая литература
1,https://www.biblio-globus.ru/product/10860521,https://static1.bgshop.ru/imagehandler.ashx?wi...,Гоголь Н.В.,Миргород,«Миргород» — цикл повестей великого классика р...,Классическая литература
2,https://www.biblio-globus.ru/product/10907351,https://static1.bgshop.ru/imagehandler.ashx?wi...,Лухманова Н.А.,Девочки,Домашняя и институтская жизнь девочек дореволю...,Классическая литература
3,https://www.biblio-globus.ru/product/10922026,https://static1.bgshop.ru/imagehandler.ashx?wi...,Катаев В.П.,Сын полка. Белеет парус одинокий,Грохочет самая страшная война в истории нашей ...,Классическая литература
4,https://www.biblio-globus.ru/product/10905935,https://static1.bgshop.ru/imagehandler.ashx?wi...,Пушкин А.С.,Капитанская дочка,Творчество Александра Сергеевича Пушкина знако...,Классическая литература
...,...,...,...,...,...,...
57,https://www.biblio-globus.ru/product/10934261,https://static1.bgshop.ru/imagehandler.ashx?wi...,Пушкин А.С.,Барышня-крестьянка.,Барышня-крестьянка.,Классическая литература
58,https://www.biblio-globus.ru/product/10915428,https://static1.bgshop.ru/imagehandler.ashx?wi...,Тургенев И.С.,Отцы и дети,"Тема отцов и детей стара как мир и, тем не мен...",Классическая литература
59,https://www.biblio-globus.ru/product/10905904,https://static1.bgshop.ru/imagehandler.ashx?wi...,Достоевский Ф.М.,Братья Карамазовы,«Братья Карамазовы» – одна из немногих в миров...,Классическая литература
60,https://www.biblio-globus.ru/product/10922026,https://static1.bgshop.ru/imagehandler.ashx?wi...,Катаев В.П.,Сын полка. Белеет парус одинокий,Грохочет самая страшная война в истории нашей ...,Классическая литература


In [ ]:
# page=requests.get('https://www.biblio-globus.ru/catalog/category?id=226&page=1&sort=0&instock=&isdiscount=')
# soup=BeautifulSoup(page.text, 'html.parser')


In [ ]:
# pg=soup.find_all('a',attrs={'aria-label':'Next'})[-1]['href']

In [ ]:
# test='https://www.biblio-globus.ru/catalog/category?id=752&page=2&sort=0&instock=&isdiscount='
# test.replace(f'id={752}',f'id=543')

'https://www.biblio-globus.ru/catalog/category?id=543&page=2&sort=0&instock=&isdiscount='

In [ ]:
# pd.Series(mega_genres_url).to_csv('test.csv',mode='a',index=False,header=False)